### OHLC DATA OF OPTIONS 
### 7:30 to 12:30 next day
### save this data to one put sheets and one call sheet then add them to main sheet 
### Column should have dates of expiry , row 1 should be time of closing and values to be added in them 


In [3]:
import requests
import pandas as pd
import datetime
from datetime import timezone



def convert_to_utc_unix(dt):
    """Converts a datetime object to UTC and then to Unix timestamp."""
    utc_dt = dt.astimezone(timezone.utc)
    unix_time = int(utc_dt.timestamp())
    return unix_time

# Assuming your start_time and end_time are already datetime objects
start_time = datetime.datetime(2024, 7, 1, 18, 30, 0)
end_time = datetime.datetime(2024, 7, 1, 20, 30, 0)

# Convert to UTC and then to Unix timestamp
unix_start = convert_to_utc_unix(start_time)
unix_end = convert_to_utc_unix(end_time)

print(unix_start)
print(unix_end)
symbol = "C-BTC-61800-020724"
resolution = "15m"
params = {
    'resolution': resolution,
    'symbol': symbol,
    'start': f"{unix_start}",
    'end': f"{unix_end}"
}
response = requests.get("https://cdn.india.deltaex.org/v2/history/candles", params=params)

# Extract the 'result' key which contains the data
data = response.json()['result']
close_data = [{'time': entry['time'], 'close': entry['close']} for entry in data]
print(data)
# Create a DataFrame
# df = pd.DataFrame(close_data)
# df['time'] = pd.to_datetime(df['time'], unit='s')
# df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
# df['time'] = df['time'].dt.strftime('%H:%M:%S')

# Prepare the data for Excel
# time_values = df['time'].tolist()
# close_values = df['close'].tolist()

# # Create a new DataFrame for the Excel format
# df_excel = pd.DataFrame(columns=['EXPIRY DATE'] + time_values)
# df_excel.loc[1, 'EXPIRY DATE'] = end_time.strftime('%Y-%m-%d')
# for i, close in enumerate(close_values):
#     df_excel.loc[1, time_values[i]] = close

# # with pd.ExcelWriter('historical_data_o.xlsx', engine='xlsxwriter') as writer:
# #     df_excel.to_excel(writer, sheet_name='Calls', index=False)
# print(df)
print("Data saved to historical_data_o.xlsx")


1719838800
1719846000
[{'close': 1181, 'high': 1181, 'low': 1181, 'open': 1181, 'time': 1719845100, 'volume': 43}, {'close': 1046, 'high': 1046, 'low': 1046, 'open': 1046, 'time': 1719844200, 'volume': 0}, {'close': 1046, 'high': 1046, 'low': 1046, 'open': 1046, 'time': 1719843300, 'volume': 0}, {'close': 1046, 'high': 1046, 'low': 1046, 'open': 1046, 'time': 1719842400, 'volume': 0}, {'close': 1046, 'high': 1084, 'low': 1046, 'open': 1084, 'time': 1719841500, 'volume': 11}, {'close': 1109, 'high': 1109, 'low': 1109, 'open': 1109, 'time': 1719840600, 'volume': 0}, {'close': 1109, 'high': 1109, 'low': 1077, 'open': 1077, 'time': 1719839700, 'volume': 16}, {'close': 1201, 'high': 1201, 'low': 1201, 'open': 1201, 'time': 1719838800, 'volume': 10}]
Data saved to historical_data_o.xlsx


In [6]:
import requests
import math 
def get_historical_atm(unix_start,resolution,index):
        start = unix_start - 3600
        end = unix_start 
        resolution = resolution
        params = {
        'resolution': resolution,
        'symbol': f"{index}USD",
        'start': f"{start}",
        'end': f"{end}"
        }
        response = requests.get("https://cdn.india.deltaex.org/v2/history/candles", params=params)
        data = response.json()['result']
        close = data[1]['close']    
        atm_value =  int(math.floor(close / 200.0)) * 200
        return atm_value 

get_historical_atm(1719763200,'1h',"BTC")     

61600

In [6]:
import datetime
from datetime import timedelta
start_time = datetime.datetime(2024, 6, 30, 21, 30, 0)
end_time = start_time + datetime.timedelta(hours=12)
print(end_time.strftime('%d%m%y'))
utc_dt = "08/04/2022 12:00:00"



010724


AttributeError: module 'datetime' has no attribute 'strptime'

In [2]:
import datetime

def date(start_time_date):
    # Parse the input string into a datetime object
    date_obj = datetime.datetime.strptime(start_time_date, "%d-%m-%Y %I:%M %p")
    
    # Construct the start_time with the specified time
    start_time = datetime.datetime(date_obj.year, date_obj.month, date_obj.day, date_obj.hour, date_obj.minute, date_obj.second)
    
    # Calculate end_time as 12 hours after start_time
    end_time = start_time + datetime.timedelta(hours=12)
    
    return start_time, end_time

# Example usage
start, end = date("02-08-2024 9:30 PM")
print("Start Time:", start)
print("End Time:", end)


Start Time: 2024-08-02 21:30:00
End Time: 2024-08-03 09:30:00


### GET INDEX PRICES 

In [ ]:
import requests
import pandas as pd
import datetime
from datetime import timezone

from pymongo import MongoClient

def get_ohlc_optiondata( index, resolution, start_time_date, hours, collection):
    
    
    def convert_to_utc_unix(dt):
        """Converts a datetime object to UTC and then to Unix timestamp."""
        utc_dt = dt.astimezone(timezone.utc)
        unix_time = int(utc_dt.timestamp())
        return unix_time

    def dates(start_time_date):
        """Parses the input string into a datetime object."""
        date_obj = datetime.datetime.strptime(start_time_date, "%d-%m-%Y %I:%M %p")
        starttime = datetime.datetime(date_obj.year, date_obj.month, date_obj.day, date_obj.hour, date_obj.minute, date_obj.second)
        return starttime
    
    start_time = dates(start_time_date)
    end_time = start_time + datetime.timedelta(hours=hours)
    
    unix_start = convert_to_utc_unix(start_time)
    unix_end = convert_to_utc_unix(end_time)

    
    # symbol = f"{optiontype}-{index}-{atm_value}-{end_time.strftime('%d%m%y')}"
    symbol = f"{index}USD"
    params = {
        'resolution': resolution,
        'symbol': symbol,
        'start': f"{unix_start}",
        'end': f"{unix_end}"
    }
    response = requests.get("https://api.delta.exchange/v2/history/candles", params=params)

    data = response.json().get('result', [])
    if not data:
        print(f"No data returned for {symbol} from {start_time_date}")
        return  # Skip the rest if there's no data
    
    close_data = [{'time': entry['time'], 'close': entry['close']} for entry in data if 'time' in entry and 'close' in entry]

    if not close_data:
        print(f"No valid data found for {symbol} from {start_time_date}")
        return
    
    df = pd.DataFrame(close_data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
    df['time'] = df['time'].dt.strftime('%H:%M:%S')

    date_data = {row['time']: row['close'] for _, row in df.iterrows()}
    document = {
        "Date": start_time.strftime('%d-%m-%y'),
        
        "data": date_data
    }
    collection.insert_one(document)
    
    print(f"Data for {start_time_date} saved to MongoDB.")

# MongoDB setup
client = MongoClient('mongodb://localhost:27017/')
db = client['ohlc_data']
## be very carefull here 
collection = db['ETHUSD 2022-2024']

# Looping the function get_ohlc_optiondata for the next 30 days
start_date = '01-04-2022 06:00 PM'
end_date = '30-04-2024 06:00 PM'

current_date = datetime.datetime.strptime(start_date, "%d-%m-%Y %I:%M %p")
end_date = datetime.datetime.strptime(end_date, "%d-%m-%Y %I:%M %p")

while current_date <= end_date:
    start_time_date = current_date.strftime("%d-%m-%Y %I:%M %p")
    get_ohlc_optiondata( 'ETH', '2h', start_time_date, 14, collection)
    current_date += datetime.timedelta(days=1)





In [27]:
import pandas as pd

df = pd.read_excel('D:/codeputs/ongoing/crypto_option_deltaexchange/server/data/EVENTS_ANALYSIS.xlsx')
btc_df = pd.read_excel('D:/codeputs/ongoing/crypto_option_deltaexchange/server/data/ohlc_data.BTCUSD 2020-2024.xlsx')

# btc_df['Date'] = pd.to_datetime(btc_df['Date'], format='%d-%m-%y')
df['Date'] = df['Date'].astype('datetime64[ns]')


In [1]:
df

NameError: name 'df' is not defined